# Домашняя работа

Будем работать с датасетом подержанных машин https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes.

Задача - построить прогноз цены машины по ее данным (год выпуска, пробег, расход, размер двигателя, тип коробки передач и тд). 

## Easy

Построить прогноз по одной марке машин при помощи решающих деревьев, взять только числовые признаки. В качестве метрики взять `r2_score`. Оценить важность признаков, проинтерпретировать модель.

## Normal

Объединить в один датафрейм данные по всем маркам машин. Преобразовать категориальные признаки.

Построить еще несколько моделей, используя подбор гиперпараметров. Сравнить между собой все построенные модели.

## Hard

Собрать датасет по одной модели на любом агрегаторе объявлений (модель должна быть из датасета). Сравнить какие признаки наиболее важны на нашем рынке, по сравнению с рынком в UK.

Сервисы не любят парсеры, поэтому скорее всего в автоматическом режиме не получится распарсить много данных. По-крайней мере, у меня не получилось как-то быстро обойти капчу. Поэтому:

- Можно вручную вбить в эксельку данные по нескольким десяткам машин
- Можно скачать несколько html и написать скрипт для их парсинга (можно использовать beautiful soup) - в таком подходе ожидаю больше сотни примеров
- Можно найти агрегатор без капчи или апишку (примеры - телеграм каналы, VK API) и собрать данные оттуда

# Easy

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
import pandas as pd

In [2]:
data = pd.read_csv('../data/toyota.csv')
data.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,GT86,2016,16000,Manual,24089,Petrol,265,36.2,2.0
1,GT86,2017,15995,Manual,18615,Petrol,145,36.2,2.0
2,GT86,2015,13998,Manual,27469,Petrol,265,36.2,2.0
3,GT86,2017,18998,Manual,14736,Petrol,150,36.2,2.0
4,GT86,2017,17498,Manual,36284,Petrol,145,36.2,2.0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6738 entries, 0 to 6737
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         6738 non-null   object 
 1   year          6738 non-null   int64  
 2   price         6738 non-null   int64  
 3   transmission  6738 non-null   object 
 4   mileage       6738 non-null   int64  
 5   fuelType      6738 non-null   object 
 6   tax           6738 non-null   int64  
 7   mpg           6738 non-null   float64
 8   engineSize    6738 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 473.9+ KB


In [4]:
data.describe()

,year,price,mileage,tax,mpg,engineSize
count,6738.000000,6738.000000,6738.000000,6738.000000,6738.000000,6738.000000
mean,2016.748145,12522.391066,22857.413921,94.697240,63.042223,1.471297
std,2.204062,6345.017587,19125.464147,73.880776,15.836710,0.436159
min,1998.000000,850.000000,2.000000,0.000000,2.800000,0.000000
25%,2016.000000,8290.000000,9446.000000,0.000000,55.400000,1.000000
50%,2017.000000,10795.000000,18513.000000,135.000000,62.800000,1.500000
75%,2018.000000,14995.000000,31063.750000,145.000000,69.000000,1.800000
max,2020.000000,59995.000000,174419.000000,565.000000,235.000000,4.500000


In [5]:
numerical_features = ['year', 'mileage', 'tax', 'mpg', 'engineSize']
numerical_features

['year', 'mileage', 'tax', 'mpg', 'engineSize']

In [6]:
X = data[numerical_features]
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
print('R2 score: ', r2)

R2 score:  0.935251109286612


In [7]:
importance_list = sorted(zip(X.columns, model.feature_importances_), key=lambda x: x[1], reverse=True)
print('Feature Importance:')
for feature, importance in importance_list:
    print(f'{feature}: {importance}')

Feature Importance:
engineSize: 0.5845113506752558
year: 0.26529011393833407
mpg: 0.08071645557914954
mileage: 0.05116547080221299
tax: 0.018316609005047495


Наиболее важный признак -- 'year' (год выпуска). Второй по значимости -- 'engineSize' (рабочий объем двигателя). Значимость остальных признаков сильно меньше.
Получили R^2 = 0.935 из коробки:) Кстати, если обучить модельку без признака 'tax' (который имеет минимальное значение важности), R^2 получится даже чуть лучше.

# Normal

In [8]:
import os
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.model_selection import RandomizedSearchCV

In [9]:
data_path = '../data/'
all_data = []
for filename in os.listdir(data_path):
    if filename.endswith('.csv'):
        path = os.path.join(data_path, filename)
        all_data.append(pd.read_csv(path))
data = pd.concat(all_data, ignore_index=True)
data.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£)
0,Corsa,2018,7885,Manual,9876,Petrol,145.0,55.4,1.4,NaN
1,Corsa,2019,11995,Manual,2500,Petrol,145.0,54.3,1.4,NaN
2,Corsa,2017,9777,Automatic,9625,Petrol,145.0,47.9,1.4,NaN
3,Corsa,2016,8500,Manual,25796,Petrol,30.0,55.4,1.4,NaN
4,Corsa,2019,10000,Manual,3887,Petrol,145.0,43.5,1.4,NaN


In [10]:
data['fuelType'].unique()

array(['Petrol', 'Diesel', 'Other', 'Hybrid', 'Electric'], dtype=object)

In [11]:
categorical_features = ['model', 'transmission', 'fuelType']

data_encoded = data.copy()
encoded_features = pd.get_dummies(data[categorical_features], drop_first=True)
data_encoded.drop(columns=categorical_features, inplace=True)
data_encoded = pd.concat([data_encoded, encoded_features], axis=1)

data_encoded.head()

,year,price,mileage,tax,mpg,engineSize,tax(£),model_ 2 Series,model_ 3 Series,model_ 4 Series,...,model_200,model_220,model_230,transmission_Manual,transmission_Other,transmission_Semi-Auto,fuelType_Electric,fuelType_Hybrid,fuelType_Other,fuelType_Petrol
0,2018,7885,9876,145.0,55.4,1.4,NaN,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,2019,11995,2500,145.0,54.3,1.4,NaN,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,2017,9777,9625,145.0,47.9,1.4,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2016,8500,25796,30.0,55.4,1.4,NaN,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,2019,10000,3887,145.0,43.5,1.4,NaN,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [12]:
data_encoded.isna().sum(axis=0)

year                          0
price                         0
mileage                       0
tax                       14213
mpg                        9353
                          ...  
transmission_Semi-Auto        0
fuelType_Electric             0
fuelType_Hybrid               0
fuelType_Other                0
fuelType_Petrol               0
Length: 208, dtype: int64

In [13]:
X = data_encoded.drop('price', axis=1)
X.drop(['tax', 'mpg', 'tax(£)'], axis=1, inplace=True)
y = data_encoded['price']

# дропаю признаки 'tax' и 'mpg', тк для модели выше они имели совсем небольшое значение
# (предполагаю, что ничего не поменяется сильно, но если результат не устроит, то можно включить их обратно)

In [14]:
X.isna().sum()

year                      0
mileage                   0
engineSize                0
model_ 2 Series           0
model_ 3 Series           0
                         ..
transmission_Semi-Auto    0
fuelType_Electric         0
fuelType_Hybrid           0
fuelType_Other            0
fuelType_Petrol           0
Length: 204, dtype: int64

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [10, 15, 20],
    'min_samples_leaf': [2, 4, 6]
}

In [16]:
rf_model = RandomForestRegressor(random_state=42)
random_search = RandomizedSearchCV(rf_model, param_distributions=param_grid, n_iter=5, cv=5, scoring='r2')

random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=5,
                   param_distributions={'max_depth': [10, 15, 20],
                                        'min_samples_leaf': [2, 4, 6],
                                        'n_estimators': [50, 100, 150, 200]},
                   scoring='r2')

In [17]:
best_rf_model = random_search.best_estimator_
y_pred = best_rf_model.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f'Best parameters: {random_search.best_params_}, R2 score: {r2}')

Best parameters: {'n_estimators': 50, 'min_samples_leaf': 2, 'max_depth': 15}, R2 score: 0.9165602410804082


Сделали перебор параметров. У лучшей модели получился R^2 = 0.917. Только на числовых признаках было чуть больше.